In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [2]:
data_path = r"C:\Users\ASUS\OneDrive - Royal University of Phnom Penh\Ai S1\Lab1\data"


In [32]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Preprocessing and augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,          # normalize
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2      # 20% validation
)

train_generator = train_datagen.flow_from_directory(
    'data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',      # since mask/no-mask is binary
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'data',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)


Found 9301 images belonging to 2 classes.
Found 2324 images belonging to 2 classes.


In [46]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2

img_size = 224
batch_size = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

train_gen = train_datagen.flow_from_directory(
    "data",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset="training"
)

val_gen = train_datagen.flow_from_directory(
    "data",
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset="validation"
)

base = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224,224,3))
base.trainable = False

model = Sequential([
    base,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])


Found 9301 images belonging to 2 classes.
Found 2324 images belonging to 2 classes.


In [49]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20
)


Epoch 1/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 217s 733ms/step - accuracy: 0.9404 - loss: 0.1591 - val_accuracy: 0.6601 - val_loss: 1.8037
Epoch 2/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 205s 704ms/step - accuracy: 0.9788 - loss: 0.0633 - val_accuracy: 0.6579 - val_loss: 1.9929
Epoch 3/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 204s 701ms/step - accuracy: 0.9817 - loss: 0.0536 - val_accuracy: 0.6588 - val_loss: 2.2814
Epoch 4/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 204s 702ms/step - accuracy: 0.9840 - loss: 0.0454 - val_accuracy: 0.6601 - val_loss: 2.3197
Epoch 5/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 204s 700ms/step - accuracy: 0.9852 - loss: 0.0447 - val_accuracy: 0.6605 - val_loss: 2.4925
Epoch 6/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 201s 691ms/step - accuracy: 0.9848 - loss: 0.0444 - val_accuracy: 0.6601 - val_loss: 2.4587
Epoch 7/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 173s 592ms/step - accuracy: 0.9854 - loss: 0.0437 - val_accuracy: 0.6609 - val_loss: 2.3350
Epoch 8/20
291/291 ━━━━━━━━━━━━━━━━━━━━ 170s 584ms/step - accuracy: 0.9849 -

In [50]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()


In [51]:
model.input_shape


(None, 224, 224, 3)

In [52]:
model.save("mask_detector_model.h5")



In [56]:
from tensorflow.keras.models import load_model

model = load_model(r"C:\Users\ASUS\OneDrive - Royal University of Phnom Penh\Ai S1\Lab1\mask_detector_model.h5")


In [64]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model_path = r"C:\Users\ASUS\OneDrive - Royal University of Phnom Penh\Ai S1\Lab1\mask_detector_model.h5"
model = load_model(model_path)

# Test image path (use raw string r"" or forward slashes)
img_path = r"C:\Users\ASUS\OneDrive - Royal University of Phnom Penh\Ai S1\Lab1\test_img4.jpg"

# Load image
img = cv2.imread(img_path)
if img is None:
    print("Image not found! Check the path.")
else:
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (224, 224))
    img_resized = img_resized.astype("float32") / 255.0  # normalize
    img_resized = np.expand_dims(img_resized, axis=0)

    pred = model.predict(img_resized, verbose=0)[0][0]
    print("Prediction value:", pred)

    if pred > 0.5:
        print("Mask On")
    else:
        print("Mask Off")


Prediction value: 3.4271987e-05
Mask Off


In [60]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from playsound import playsound
import time
import os

# -----------------------------
# Load trained model
# -----------------------------
model_path = r"C:\Users\ASUS\OneDrive - Royal University of Phnom Penh\Ai S1\Lab1\mask_detector_model.h5"
model = load_model(model_path)

# Haar Cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Alert sound
alert_file = r"C:\Users\ASUS\OneDrive - Royal University of Phnom Penh\Ai S1\Lab1\mask_alert.wav"

# Alert settings
last_alert_time = 0
alert_delay = 2  # seconds between alerts

# Prediction threshold
mask_threshold = 0.5  # adjust based on your model

# Open webcam
cap = cv2.VideoCapture(0)
cv2.namedWindow("Face Mask Detector", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Face Mask Detector", 800, 600)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.1, 5, minSize=(60,60))

        if len(faces) == 0:
            last_alert_time = 0  # reset timer if no faces

        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            face_rgb = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
            face_resized = cv2.resize(face_rgb, (224, 224))
            face_resized = face_resized.astype("float32") / 255.0  # normalize
            face_resized = np.expand_dims(face_resized, axis=0)

            pred = model.predict(face_resized, verbose=0)[0][0]
            print("Pred:", pred)

            if pred > mask_threshold:
                label = "Mask On"
                color = (0, 255, 0)
            else:
                label = "Mask Off"
                color = (0, 0, 255)
                # Play alert
                current_time = time.time()
                if current_time - last_alert_time > alert_delay:
                    if os.path.exists(alert_file):
                        playsound(alert_file)
                    last_alert_time = current_time

            # Draw rectangle and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        cv2.imshow("Face Mask Detector", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except KeyboardInterrupt:
    print("Stopped by user")

finally:
    cap.release()
    cv2.destroyAllWindows()


Pred: 0.99999994
Pred: 0.9999998
Pred: 0.9999985
Pred: 0.9999993
Pred: 0.9999999
Pred: 1.0
Pred: 0.99999994
Pred: 0.9999999
Pred: 0.99999994
Pred: 0.9999998
Pred: 0.99999964
Pred: 0.9999998
Pred: 0.9999989
Pred: 0.9999999
Pred: 0.99999976
Pred: 0.9999998
Pred: 1.0
Pred: 1.0
Pred: 0.99999994
Pred: 0.99999994
Pred: 0.9999995
Pred: 0.9999979
Pred: 0.9999999
Pred: 0.99999976
Pred: 0.99999875
Pred: 0.99999946
Pred: 0.99999934
Pred: 0.9999997
Pred: 0.9999927
Pred: 0.99998987
Pred: 0.45721328
Pred: 0.9999997
Pred: 0.99999976
Pred: 0.99999934
Pred: 0.999999
Pred: 0.9999999
Pred: 0.9999997
Pred: 0.9999998
Pred: 0.99999976
Pred: 0.99999124
Pred: 0.9999929
Pred: 0.9999999
Pred: 0.99999994
Pred: 0.99999994
Pred: 0.99999493
Pred: 0.9999981
Pred: 0.9999989
Pred: 0.99999994
Pred: 0.99999994
Pred: 0.99999946
Pred: 0.99999404
Pred: 0.9999931
Pred: 0.9999936
Pred: 0.9999968
Pred: 0.99999684
Pred: 0.9999935
Pred: 0.99998945
Pred: 0.9999966
Pred: 0.99999917
Pred: 0.99999756
Pred: 0.9999987
Pred: 0.9999999

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x000001C691B93EC0>
Traceback (most recent call last):
  File "C:\Users\ASUS\AppData\Local\Programs\Python\Python311\Lib\weakref.py", line 369, in remove
    def remove(k, selfref=ref(self)):

KeyboardInterrupt: 


Pred: 0.99999946
Pred: 0.99999946
Pred: 0.9999997
Pred: 0.99999774
Pred: 0.999965
Pred: 0.9952227
Pred: 0.9997277
Pred: 0.9999995
Pred: 0.99999994
Pred: 0.9999998
Pred: 0.99999994
Pred: 1.0
Pred: 0.9999995
Pred: 0.99999994
Pred: 0.99999994
Pred: 0.99999636
Pred: 0.9999969
Pred: 0.9999973
Pred: 0.9999999
Pred: 0.9999997
Pred: 0.99999696
Pred: 0.99991727
Pred: 0.9991198
Pred: 0.99579066
Pred: 0.99999404
Pred: 0.99995196
Pred: 0.9784438
Pred: 0.9999501
Pred: 0.9101017
Pred: 0.9928359
Pred: 0.42601094
Pred: 0.8255927
Stopped by user
